### Data Science Solution(KOR Ver)
- This is written in Korean with reference to this link
    - link : https://www.kaggle.com/startupsci/titanic-data-science-solutions/notebook

**제가 NoteBook을 작성하게 된 계기는 처음 Kaggle에 접하시거나, DataScience가 무엇인지 한번 쯤 간단하게 접해보고 싶은 사람을 위해 작성하였습니다. 또한 Kaggle에 한글로 번역된 자료가 많이 없기에 작성한것도 있습니다. 처음 번역하여 NoteBook을 작성하였는데 오타 및 수정이 필요한 부분이 있을 수 있습니다. 발견하신 분들은 commit 남겨주시면 수정하도록 하겠습니다. 또한 좋게 봐주시는 분들이 있으면 추가적으로 매주 1회 정도 competition 한가지를 번역하여 정리해서 올릴려고 합니다. 이상 긴글 읽어주셔서 감사합니다.**


## 목차
- 훈련, 테스트 데이터에 대한 습득(Acquire training and testing data)
- 데이터 준비 및 다루기(Wrangle, prepare, cleanse the data)
- 데이터 탐험, 패턴, 분석하기(Analyze, identify patterns, and explore the data.)
- 모델 생성 및 문제 해결 및 예측(Model, predict and solve the problem.)
- 문제 해결 단계와 최종 Solution, 시각화 (Visualize, report, and present the problem solving steps and final solution.)
- 결과 제출(Supply or submit the results.)


In [ ]:
# 데이터 분석 및 다루기(data analysis and wrangling)
import pandas as pd
import numpy as np
import random as rnd

# 데이터 시각화(visualization)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 머신러닝(machine learning)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


## Dataset 가져오기
- Pandas 패키지를 이용하여 dataset을 불러오겠습니다.
    - Train, Test 데이터를 DataFrame 형태로 변환후, 데이터셋을 결합하여, 동시에 작업을 수행하겠습니다.

In [ ]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')
combine = [train_df , test_df]

## 어떠한 컬럼이 범주형 데이터인가?
- 범주형 데이터 내에 명목,순서,비율,간격 기반의 범주형 데이터가 존재합니까?
- 이러한 데이터들은 시각화에 도움을 주는 그래프를 선택하는 매우 도움이 됩니다.
    - **Nominal : Survived, Sex, Embarked** 
    - **Ordinal : Pclass**   

## 어떠한 컬럼이 수치형 데이터인가?
- 수치형 데이터 내에 이산,연속,시계열에 기반인 데이터가 존재합니까?
- 이러한 데이터들은 시각화에 도움을 주는 그래프를 선택하는 매우 도움이 됩니다.
    - **Continous : Age, Fare** 
    - **Discete: SibSp, Parch**

추가로 데이터의 특징에 대한 설명은 아래 Link를 참고하기 바랍니다.
- link : https://blog.naver.com/qkrdnjsrl0628/222595486903

In [ ]:
train_df.columns.values

**Data에 대한 상세한 설명은 Titanic-data에 설명이 명시되어 있으니 확인해주시기 바랍니다.**

In [ ]:
train_df.head()

## 데이터 타입이 혼용된 컬럼은 어떤 것이 있습니까?
- Ticket 데이터는 알파벳과 수치형데이터가 섞여 있으며, Cabin 데이터 또한 알파벳과 숫자 데이터가 섞여 있다.

## 어떤 컬럼이 누락되어있거나 NULL 값을 가지고 있습니까?
- 대규모 데이터에서는 다루기가 매우 어렵습니다. 하지만 몇가지 샘플을 확인하면 어떤 컬럼을 수정해야 하는지 바로 알 수 있다.

- Name 데이터에는 대체 이름, 짧은 이름에 사용되는 따옴표를 포함하여 이름을 설명하는데 여러가지 방법이 있으므로, 오류나 오타가 포함되어 있을 수 있다.

In [ ]:
train_df.tail()

## <strong>어떤 데이터가 누락 및 NUL을 포함하고 있습니까?</strong>
- NULL값 및 누락 데이터는 수정될 필요가 있습니다.
- 훈련 데이터 내에서는 Cabin > Age > Embarked 순으로 NULL 값 데이터가 존재합니다.
- 테스트 데이터 내에는 Cabin > Age 순으로 NULl 값 데이터가 존재합니다.

## <strong>데이터 타입의 개수가 어떻게 되나요?</strong>
- 훈련 데이터에는 7개의 데이터들이 정수형 또는 실수형으로 이루어져 있으며, 테스트 데이터는 6개의 정수형 또는 실수형으로 이루어져 있다.
- 5개의 데이터 타입은 object이다.

In [ ]:
train_df.info()
print('_'*40)
test_df.info()

## <strong>수치형 데이터의 분포가 어떻게 되나요?</strong>
- <strong>수치형 데이터의 분포를 파악하는 것은 인사이트 를 얻는 것과 훈련데이터가 어떤 문제의 영역에 어떻게 직면하는 아는데 도움을 준다.<strong>
    <br></br>
    - 총 표본 데이터는 891개이다.
    - 생존 데이터의 범주는 0, 1로 이루어져 있음을 알 수 있다.
    - 생존을 나타내는 데이터가 38%임을 알 수 있다.
    - 75% 이상이 부모 또는 자녀들과 여행을 가지 않았음을 알 수 있다.
    - 승객의 30%정도는 형재 자매 및 배우자와 함께 타고 있음을 알 수 있다.
    - 요금을 512달러까지 지불하는 승객들도 존재합니다.
    - 65 - 80세 사이의 나이 많은 어르신분들이 대략 1%존재한다는 것을 알 수 있습니다.

In [ ]:
train_df.describe()

## <strong>범주형 데이터는 어떻게 분포되어 있습니까?</strong>
* 참고로 DataFrame.describe() 을 사용하면 수치형 데이터만을 알려주지만,&nbsp;
  <strong>include = ['O'] 를 지정하면, 범주형 데이터를 알려준다.</strong>
  
  - Name 데이터는 유니크 데이터로 891개 존재한다.
  - 성별의 65%는 남자임을 알 수 있다.
  - Cabin 데이터 내에는 여러 데이터에 거쳐 중복항목이 존재한다. 몇몇 승객들은 선실을 공유하였습니다.
  - Embarked 데이터에는 3가지 데이터가 존재한다. 대부분의 사람들이 S 라는 데이터를 가지고 있습니다.
  
  
  

In [ ]:
train_df.describe(include = ['O'])
# include 0으로 할 경우 문자열 데이터에 관한 통계를 알려준다.

## <strong>데이터 분석을 기반으로 한 가정</strong>
- 데이터를 가지고 적절한 행동을 하기전 이러한 가정들을 검증할 것입니다.

#### Correlating(상관 분석)
- 나이와 생존간의 관계는 명확한 상관관계가 존재한다.
- 생존 또는 또 다른 중요한 특징과 탑승지역과 관련지을 수도 있습니다.

#### Correcting(정확성)
- Ticket 데이터는 생존과 관계가 없을 뿐만 아니라, 22%의 데이터가 중복되므로 삭제하는 것이 좋습니다.
- Cabin 데이터는 불완전하거나 많은 NULL값을 가지고 있기에 훈련 테스트 데이터에서 drop하는 것이 좋습니다.
- PassengerID는 Survived와 관련이 없으므로, 삭제하는 것이 좋습니다.
- Name 도 위와 동일합니다.

#### Creating(제작)
- Parch, Sibsp 기반으로 하여 가족의 총 수를 얻을 수 있습니다.
- 새로운 특징을 추출하기 위해 Name 데이터를 다룰 수 있습니다.
- Age 데이터를 가지고, 새로운 범주형 데이터를 만들수 있습니다.
- Fare 또한 분석에 도움이 된다면 범주형으로 변경할 수 있습니다.


#### Classifying(분류)
- 문제의 설명을 기반으로 가정을 추가할 수 있습니다.
- Wonme 는 생존할 가능성이 더 있습니다.
- Age가 낮을수록 더욱 생존할 가능성이 존재합니다.
- Pclass = 1 에 가까울수록 생존 할 가능성이 큽니다.

## <strong>Analyze by pivoting features</strong>

- 몇몇 우리의 검증과 추정을 확증하기 위해, 특징을 pivot 함으로써 상관 관계를 신속하게 분석할 수 있습니다.
- 이 단계는 공백의 데이터가 존재하지 않는 것에 한에서만 시행이 가능하다.
- 데이터들을 pivoting 하는 것은 범주형 데이터나 이산형 데이터에 한해서만 시행하는 것이 좋다.

    - Pclass : Pclass = 1 과 Survived 간의 상관관계가 있다는 것을 파악했으며, 모델의 특징에 포함하기로 한다.
    - Sex : Female인 경우 생존률이 74% 이상임을 파악할 수 있습니다.
    - SibSp and Parch : 값에 대하여 상관관계가 0이므로, 기능집합을 파생하는 것이 가장 좋다.
    

In [ ]:
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)
# as_index 는 Pclass 를 그룹으로 한 것을 index화를 하는 것이냐, 안하는 것이냐를 지정하는 것이다.

In [ ]:
train_df[['Sex', 'Survived']].groupby(['Sex'], as_index = False).mean().sort_values(by='Survived', ascending = False)

In [ ]:
train_df[['SibSp', 'Survived']].groupby(['SibSp'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
train_df[['Parch', 'Survived']].groupby(['Parch'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

## <strong>데이터 시각화를 통한 분석</strong>
- 분석한 데이터를 시각화 함으로써 우리의 몇몇 가정들을 검증해 나갈것입니다.
## 수치형 데이터 관계성
    - 수치형 데이터와 타겟데이터와의 관계를 이해해보려 하겠습니다.
        - history chart 는 매우 유용하다. 연속적인 수치형 데이터의 유용한 패턴을 분석하는데 유용하다.
        - histogram은 막대를 사용함으로써 데이터가 나타난 분포를 횟수를 그래프로 나타낸 것입니다.
        - histogram 관련한 설명 링크 : https://sohyunwriter.tistory.com/142?category=892943
        - histogram은 어릴수록 생존률이 높은지에 관하여 관련된 시각을 보여준다.
        
#### Observations
- 4세 이하는 높은 생존률을 나타낸다.
- 나이가 많을수록 생존률이 높다.
- 15-25세 사이의 대부분은 생존하지 못하였다.
- 대부분의 승객들은 15- 35세이다.

#### Decisions
- 이러한 간단한 분석은 차후 흐름에 대한 결정으로 우리의 가정을 확인한다.
    - 모델에서 나이를 고려해야 함을 알 수 있다. (classifying 2)
    - NULL 연령 기능을 처리해야 한다. (completing 1)
    - 연령대별로 묶어야 함을 알 수 있다. (creating 3)
      


    

In [ ]:
# FaceGrid(참고) :  https://steadiness-193.tistory.com/201
g = sns.FacetGrid(train_df, col = 'Survived')
g.map(plt.hist, 'Age', bins = 20)

## 수치형 데이터와 순서형 데이터 간의 상관관계
- 단일 그래프를 사용함으로써 상관관계를 식별하기 위해 여러 기능을 결합할 수 있다.
- 숫자 및 범주 기능으로써 사용할 수 있습니다.

#### Observations
- 대부분의 승객들은 Pclass = 3, 하지만 죽지는 않았다. (classifying 2)
- Pclass = 2 or Pclass = 3 인 유아기의 아이들은 대부분 생존했다. (classifying 2)
- Pclass = 1인 대부분의 승객들은 생존하였다. (classifying 3)
- Pclass 는 승객의 연령에 따라 역할이 다름을 파악할 수 있다.

#### Decisions
- 모델을 학습할 때, Pclass를 고려할 수 있다.

In [ ]:
grid = sns.FacetGrid(train_df, col = 'Pclass',hue = 'Survived')
grid.map(plt.hist, 'Age', alpha = .5, bins = 20)
grid.add_legend()

In [ ]:
grid = sns.FacetGrid(train_df, col = 'Survived', row = 'Pclass', size = 2.2, aspect = 1.6)
grid.map(plt.hist, 'Age', alpha =.5, bins = 20)
grid.add_legend()

## 범주형 데이터 상관관계
- Solution을 위해 범주형 데이터의 상관관계를 파악할 수 있다.

## Observations.
- 여성 승객은 남성보다는 높은 생존률을 지니고 있음을 알 수 있다.(classifying 1)
- 남성이 더 높은 생존률을 보인 Embarked = C는 예외임을 알수 있고,Pclass와  Survived 관점에서 Pclass와 Embarked간의 상관관계가 있음을 알 수 있다. 단, Embarked 와 Survived는 직접적인 상관관계가 필수조건은 아님을 알 수 있다.
- 남성의 경우 Pclass = 3 이 Pclass = 2 보다 생존률이 높음을 알 수 있다. (Completing 2)
- Pclass = 3의 경우 특성이 매우 다양함을 알 수 있다. (Completing 1)

## Decisions
- 성별의 경우 모델을 학습시킬때 추가해야함을 알 수 있다.
- 모델 학습을 완료한후 Embarked 특성을 추가한다.

In [ ]:
grid = sns.FacetGrid(train_df, col = 'Embarked')
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette = 'deep')
grid.add_legend()

In [ ]:
grid = sns.FacetGrid(train_df, row = 'Embarked', size = 2.2, aspect = 1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette = 'deep')
grid.add_legend()

## 범주형 데이터와 수치형 데이터의 상관관계
- 범주형 데이터와 수치형 데이터의 상관관계를 파악해야 한다. 우리는 범주형 데이터인 Embarked, Sex, Survived와 수치형 데이터인 Fare를 비교할 수 있다.

## Observations
- 높은 요금을 지불한 사람들은 생존률이 높을 알 수 있다. (creating 4)
- 승선항과 생존률간에 관계가 있음을 알 수 있다.

## Decision
- 요금을 범위별로 묶을 수 있다.


In [ ]:
grid = sns.FacetGrid(train_df, row = 'Embarked', col = 'Survived', size = 2.2, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha = .5, ci = None)
# alpha 는 투명도(transparency)를 의미합니다.
grid.add_legend()

In [ ]:
grid = sns.FacetGrid(train_df, col = 'Embarked', hue = 'Survived',  palette={0: 'k', 1: 'w'})
grid.map(sns.barplot, 'Sex', 'Fare', alpha = .5, ci = None)
# alpha 는 투명도(transparency)를 의미합니다.
grid.add_legend()

## 데이터 전처리(Preprocessing Data)
- 본문에는 Wrangle Data라고 작성되어 있는데 데이터 전처리랑 비슷한 의미이다.
- 참고 : https://bkshin.tistory.com/entry/DATA-23-Data-Wrangling

- 데이터와 특정 솔루션과 관련하여 몇몇의 결정과 가정을 수집했다. 지금까지 값이나 기능을 변경할 필요가 없었다.지금부터는 문제를 해결하기 위해 데이터를 가공하면서 결정과 가정이 맞는지 파악해보겠습니다.

## 데이터 삭제
- 데이터를 삭제함으로써 더 적은 양의 데이터를 처리한다.
- 우리의 추정과 결론을 기반으로 하여 우리는 Cabin 과 Ticket의 데이터 를 버릴 수 있다.
- 일관성을 유지하기 위해서 훈련데이터와 테스트 데이터를 동시에 작업해줘야 한다.

In [ ]:
print('Before', train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

train_df = train_df.drop(['Ticket', 'Cabin'], axis = 1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis = 1)
combine = [train_df, test_df]

print('After', train_df.shape, test_df.shape, combine[0].shape, combine[1].shape)

## 존재하는 데이터로부터 새로운 데이터 추출하기
- 만약 Name,Paseengerid 데이터를 삭제하기전에 Name 이름에서 성(Title)과 Survived간의 상관관계를   파악할 수 있도록 분석할 수 있습니다.
- 정규표현식을 이용해서 추출할 수 있습니다.

## Observation
- 대부분의 성(Title)은 나이를 정확히 표현합니다. 예를들면 성(Title)이 Master인 사람은 평균적으로 5세 이하입니다
- 성(Title)과 Age간의 생존관계는 서로 다릅니다.
- 특정 성(Title)을 가진 사람은 사망했거나(Mme, Lady, Sir) 혹은 생존하였습니다.(Don, Rev, Jonkheer)

## Decision
- Model 훈련을 위해서 새로운 제목을 유지하기로 하였습니다.

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand = False)

pd.crosstab(train_df['Title'], train_df['Sex'])

많은것을 더 일반적인 Title으로 변경하거나 Rare로 분류할 것입니다.

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')    
    
train_df[['Title', 'Survived']].groupby(['Title'], as_index = False).mean()

범주형(categorical) 데이터인 title을 순서형(ordinal) 데이터로 변환할 것입니다.

In [ ]:
title_mapping = {"Mr":1 , "Miss":2, "Mrs":3, "Master":4, "Rare":5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)
    
train_df.head()

In [ ]:
train_df = train_df.drop(['Name', 'PassengerId'], axis = 1)
test_df = test_df.drop(['Name'], axis = 1)
combine = [train_df, test_df]
train_df.shape, test_df.shape 

## 범주형 데이터 변환
- 문자형, 수치형 데이터를 변환할 수 있습니다. 대부분의 모델에서는 요구되는 사항입니다.
- 먼저 female, male 데이터를 변환하겠습니다. female = 1, male = 0

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female' : 1, 'male' : 0}).astype(int)

train_df.head()

## 수치형 연속형 데이터 다루기
- 공백이 있거나 NULL값을 가진 데이터를 다루는 것부터 시작하겠습니다. 먼저 Age Feature부터 다루겠습니다.
- 3가지 방식으로 연속형 데이터를 다룰 수 있습니다.
    - 간단한 방식으로는 평균과 표준편차 사이에서 난수를 생성하는 방식이 있습니다.
    - 공백이 있는 데이터를 더 정확하게 추측하는 방법으로 다른 정확한 데이터를 사용하는 방식이 존재합니다. 나이, 성별, Pclass 사이의 관계를 주목하겠습니다. Pclass 와 Gender를 결합하여 Age 에서 mean을 찾음으로써 추측을 할 수 있습니다.
    - 첫번째 방식과 두번째 방식을 결합하는 것이 좋다. 따라서 Age를 중앙값으로 추측하는 대신 Pclass 및 Gender 조합 집합을 기반으로 평균 및 표준편차 사이의 난수를 사용할 것입니다.
    
    - random은 노이즈를 도입할 수 있습니다. 하지만 실행 결과가 모두 다를 수 있으므로. 2번째 방법을 이용하겠습니다.
    

In [ ]:
grid = sns.FacetGrid(train_df, row = 'Pclass', col = 'Sex')
grid.map(plt.hist, 'Age')
grid.add_legend()

In [ ]:
guess_ages = np.zeros((2, 3))
guess_ages

지금부터 Sex(0 or 1) 그리고 Pclass(1, 2, 3)을 조합하여 6번의 반복으로 Age 데이터를 추측할 것입니다.

In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & (dataset['Pclass'] == j+1)]['Age'].dropna()
            
            age_guess = guess_df.median()
            
            guess_ages[i,j] = int((age_guess/0.5) + 0.5)*0.5
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1), 'Age'] = guess_ages[i, j]
    
    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

Age 데이터를 Survived와 관련하여 묶어서 확인해보겠습니다

In [ ]:
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index = False).mean().sort_values(by = 'AgeBand', ascending = True)

In [ ]:
for dataset in combine:
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age'] = 4

train_df.head()

In [ ]:
train_df = train_df.drop(['AgeBand'], axis = 1)
combine = [train_df, test_df]
train_df.head()

## 존재하는 Feature를 결합하여 새로운 Feature 만들기
- 우리는 Parch, SibSp를 결합하여 식구(가족 총 인원 수)에 대한 새로운 Feature를 생성할 수 있습니다.
- 그렇게 함으로써 Parch, SibSp데이터를 제거가 가능해집니다.

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index = False).mean()

In [ ]:
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis = 1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis = 1)

combine = [train_df, test_df]

train_df.head()

Pclass 와 Age 에 관한 데이터를 결합함으로써 새로운 Feature를 생성 수 있습니다.

In [ ]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

## 범주형 데이터 다루기
- Embarked Feature 는 S, Q, C 데이터를 가지고 있습니다. 훈련데이터에는 2개의 누락값이 존재합니다. 단순히 가장 흔히 나온 데이터로 채우겠습니다.

In [ ]:
# mode는 최빈값을 출력합니다. 참고 : https://www.geeksforgeeks.org/python-pandas-dataframe-mode/
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)

train_df[['Embarked', 'Survived']].groupby(['Embarked'], as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

## 수치형 데이터 다루기
- 최빈값을 이용함으로써 Fare요금에 대하여 누락된 값을 다룰 수 있습니다. 
- 소수 두번째 자릿수로 반올림하여 실행하겠습니다.

In [ ]:
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace = True)
test_df.head()

In [ ]:
# 참고 : https://steadiness-193.tistory.com/67
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df[['FareBand', 'Survived']].groupby(['FareBand'], as_index = False).mean().sort_values(by = 'FareBand', ascending = True)

FareBand를 기반으로 Fare feature 를 변경합니다.

In [ ]:
for dataset in combine:
    dataset.loc[dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare'] = 2
    dataset.loc[dataset['Fare']>31 , 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis = 1)
combine = [train_df, test_df]
train_df.head()

## 예측 및 문제 해결하기
- 60개 이상의 모델에서 선택을 할 수 있습니다.
- 문제의 종류를 파악해서 문제에 적합한 모델로 해결해야 합니다.

## 모델
- Logistic Regression
- KNN or K-Nearst Neigbors
- Support Vector Machines
- Navie Bayes classifier
- Decision Tree
- Random Forest
- Perceptron
- Artifical neural network
- RVM or Relevance Vector Machine

In [ ]:
X_train = train_df.drop('Survived', axis = 1)
Y_train = train_df['Survived']
X_test = test_df.drop('PassengerId', axis = 1).copy()
X_train.shape , Y_train.shape, X_test.shape

Logistic Regresson는 초기 작업흐름을 파악하는 유용한 모델입니다.
- Logistic Regresson의 특징에 관해서는 아래의 링크를 참고해주시기 바랍니다.<br></br>
  https://ko.wikipedia.org/wiki/%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1_%ED%9A%8C%EA%B7%80 

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, Y_train)
Y_pred = log_reg.predict(X_test)
acc_log = round(log_reg.score(X_train, Y_train) * 100 , 2)

print(acc_log)

저희의 추정과가정을 검증하게 위해 Logistic Regression을 사용하였습니다. 결정계수를 활용한 특정 계수를 확인 하였습니다. Positive Coefficients increase 로그 승산을 증가 시키며, Negative Coefficients 는 로그 승산을 감소시킵니다.
- Sex Feature의 경우 매우 높은 양의 승산임을 알 수 있다. Survived = 1 이기 대부분이기 때문이다.
- Pclass가 증가할수록 Survived 1일 확률이 매우 감소한다는 것을 알 수 있다.
- Age*Class 는 Survived 와 음의 상관관계가 두번째로 높기에 modeling 하기에 매우 좋습니다.
- Title 은 두번째로 양의 승산임을 알 수 있습니다.

In [ ]:
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df['Correlation'] = pd.Series(log_reg.coef_[0])

coeff_df.sort_values(by = 'Correlation', ascending = False)

분류 및 회귀 분석에 사용되는 지도 학습 모델인 SVM을 사용하여 모델링 해보겠습니다.
일련의 훈련 샘플이 주어지면 SVM은 한 범주가 다른 범주에 포함되는 모델을 구축하여 이진 선형 분류기를 생성합니다.
- Logistic Regression 보다 confidence가 높습니다.

In [ ]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100 , 2)
acc_svc

KNN을 사용하여 분류와 회귀에 적용해 보겠습니다. KNN은 과반수의 투표로 구분 됩니다.
- KNN은 Logistic Regression 보다 좋지만, SVM보다 cofidence score가 낮습니다.

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train)* 100 , 2)
acc_knn

Navie Bayes를 이용해보겠습니다.
- 참고 : https://ko.wikipedia.org/wiki/%EB%82%98%EC%9D%B4%EB%B8%8C_%EB%B2%A0%EC%9D%B4%EC%A6%88_%EB%B6%84%EB%A5%98

In [ ]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

Perceptron을 이용해보겠습니다.
- 참고 : https://ko.wikipedia.org/wiki/%ED%8D%BC%EC%85%89%ED%8A%B8%EB%A1%A0

In [ ]:
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

LinearSVC을 이용해보겠습니다.
- 참고 : https://jfun.tistory.com/105

In [ ]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

SGDClassifier을 이용해보겠습니다.
- 참고 : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

In [ ]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

## Model Evaluation
- 모든 모델에 대한 rank를 매김으로 써 가장 적합한 모델을 선택해 보겠습니다.
    - Decision Tree와 Random Forest는 모두 동일한 모델이지만, Overfitting 방지를 위해서 random_forest를 이용하겠습니다.

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('submission.csv', index=False)

## 이상 긴글 읽어주셔서 감사합니다.
- 추가로 GridSearchCV로 하이퍼파라미터 최적화도 실행할 수 있지만 제가 이 Notebook을 번역한 목적은 DataScience입문하는 사람을 목적으로 하였기에 여기까지만 작성하였습니다.